# Fun time with Bayesian statistics and Politics 

In [1]:
from IPython.display import Image
Image(url='http://a.abcnews.go.com/images/Politics/GTY_donald_trump_hillary_clinton_sk_150619_16x9_992.jpg')

The presidential voting season was recently upon us, and what better way to celebrate it (lol), and our newfound understanding of Bayesian statistics by fusing the two together? In the 2012 last presidential electoral cycle, Nate Silver won acclaim (and some scorn from academia for the acclaim he garnered with work that lacked much novelty) for utilizing a Bayesian approach to his poll prediction, in particular he utilized something called a Monte Carlo Markov Chain (MCMC). We're going to start our nascent political data science careers in this lab with something slightly less ambitious, but no less relevant to the material we've gone through so far.

# Data Background 

# Some model background

#### 2.1 Priors and Conjugate Priors

### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Conjugate Priors
| Prior  | Likelihood  | 
|---|---|
 | Beta  | Binomial  |
 | Dirichlet  | Multinomial   | 
 | Gamma  | Gaussian |
 | Gaussian  | Gaussian | 
 
 The above table provides the likelihood/prior conjugate prior combinations that ensure computational tractability for calculation of posterior distributions 

We've seen some of these previously, but let's just summarize these distributions now: 

| Name  | Function (The Distribution/PMF) - Whatever| Use
|---|---|
| Beta  | $\frac{1}{B(\alpha, \beta)}x^{\alpha-1}(1-x)^{\beta-1}$ | More of a family of distributions, usually representing probabilties of proportions, thus it's great to use as our prior belief (very important in Bayesian analysis)|
| Dirichlet  |  $\begin{equation*}\frac{1}{B(\alpha, ,\beta)}\prod_{i = 1}^{n}x_i^{\alpha-1}\end{equation*}$  | Similar to Beta above, just extended over multinomials, often used in NLP and other text-mining processes  |
| Gamma  |$ x^{a-i}e^{-x}\frac{1}{\Gamma(a)}$ | This distribution represents waiting time between Poisson distributed events (remember c_i in the previous lecture finger exercise | 
| Gaussian  | $\frac{1}{\sigma \sqrt(2\pi)}e^{\frac{-(x-\mu)^{2}}{2\sigma^2}}$ | This is just the normal distribution | 
| Binomial  | $\binom{n}{k}\cdot p^kq^{n-k} $   | Gives you the probability of getting k "success" in n iterations/trials |
| Multinomial  | $\frac{N!}{\prod_{i=1}^n x_i!} \prod_{i=1}^n \theta_i^{x_i} $| Similar to binomial distribution, but generalized to include multiple "buckets" (X, Y, Z,...) instead of just two (1 or 0), represents how many times each bucket X, Y, Z,... etc was observed over n trials | 

#### 2.2 Beta functions as normalizing term in case you don't want to make your life easy and use conjugate priors...

We are finally going to use the much anticipated Beta Function today to do our first real computational Bayesian exercise, $\frac{1}{B(\alpha, \beta)}x^{\alpha-1}(1-x)^{\beta-1}$. Wait, what's B? We can define B as the following: 
$$B(a,b) = \frac{\Gamma(a)\Gamma(b)}{\Gamma(a+b)}$$

As it is used in Bayesian Analysis, the beta distribution (not "B" - or the beta function) is often used to represent the prior beliefs of the modeler. How so? Well let's say you thought the chance of a political candidate winning an election was .60 (whatever that means), but that given new information, that you've cleverly modeled using Bayes, it's now .80 (say you've tied your candidates political fortunes to the performance of the economy), your prior beliefs can be modeled (after some tuning of $\alpha$, $\beta$, the input variable) by the beta distribution.

**The beta function** in contrast, is often used as a scaling term that ensures your model is bounded by (0, 1).
 

Build the Beta function into Python using the math library

In [1]:
import math
from scipy.special import gamma as Gamma
from scipy.stats import beta

def my_beta(a,b):
    beta_num = (math.gamma(a)*math.gamma(b))/math.gamma(a+b)
    return beta_num

#### 2.3 (An advanced preview) Introducing Dirchilet Distribution for Multi-level Categorical Analysis

In the longer term, our main aim is to model the quotient, in particular, the percentage of voters for a particular candidate. The 'commonest' way to do this (to borrow a phrase from our British friends), is to utilize the Dirchilet distribution in our probability model: $\begin{equation*}\frac{1}{B(\alpha)}\prod_{i = 1}^{n}x_i^{\alpha-1}\end{equation*}$ (stay tuned for this later)  

As you read in the summary in the table above, it's often used to model situations with more than "2" possiblities/buckets/outcomes/whatever. This is in fact a very powerful distribution. 

For our purposes, we would use this distribution to model situations where there are more than 2 candidates (Clinton, Bush, Perot in 1992, or maybe Clinton, Trump, Sanders in 2016. But for this lab,  let's assume there's just two alternatives. That way, we can utilize the simple binomial model and as our table states, we just need to use beta for that case.


# Prepping the data

This case study will be based off of the following data: http://www.stat.columbia.edu/~gelman/arm/examples/election88/, from the **great one**, Dr. Gelman, one of the foremost researchers in Bayesian analysis. 

This election was ultimately a major victory for the Grand Old Party (GOP/Republicans), and propelled George H.W. Bush (the father of George W. Bush) into the presidency. 

Import the data, and delete the Unnamed column

In [5]:
import pandas as pd

# Reading in the csv file
pre_poll = pd.read_csv('polls.csv')

# Calling a conversion method of your data to csv 
#pre_poll.to_csv('F:/Bayesian Lab/polls.csv')
print(type(pre_poll))
pre_poll.head(10)

# Further, we can remove org, as it is just one value -- Special note, in general, we probably don't want to do this, because 
# there may be a chance we want to merge/join this file with another and that column could end up being a unique identifier for 
# a subset of data in our large dataset
pre_poll['org'].value_counts()
del pre_poll['org']

pre_poll.head()

<class 'pandas.core.frame.DataFrame'>


,year,survey,bush,state,edu,age,female,black,weight
1,1,9152,1.0,7,2,2,1,0,1403
2,1,9152,1.0,33,4,3,0,0,778
3,1,9152,0.0,20,2,1,1,0,1564
4,1,9152,1.0,31,3,2,1,0,1055
5,1,9152,1.0,18,3,1,1,0,1213


Now that we have the data read in, we would like to partition the data into states so we can do state-specific inference 

Summarize the age variable with counts

In [8]:
# Determine the number of years of the data
pre_poll['age'].value_counts().head()

2    4788
3    3429
1    3149
4    2178
Name: age, dtype: int64

Summarize the state with counts

In [9]:
# Determine the number of entities/states of the data
pre_poll['state'].value_counts()


5     1493
33     894
44     788
10     750
39     616
36     605
14     567
23     530
31     428
48     393
50     389
22     373
47     354
34     346
43     329
11     316
26     309
15     291
24     289
21     284
41     223
25     220
18     210
1      203
19     196
3      194
6      181
7      171
38     149
16     143
17     141
37     130
28     125
4      121
49     117
32     109
40      91
45      79
42      60
35      60
20      51
13      42
27      40
8       39
29      32
30      27
9       19
51      15
46      12
Name: state, dtype: int64

create a data frame with only state and age

In [10]:
# We need to partition the data into individual data frames to construct our Bayesian inference on 
c1 = pre_poll['state'] == 1
c2 = pre_poll['age'] == 4
subset_interest = pre_poll[c1 & c2]


In [11]:
test_set = subset_interest.dropna(); test_set.head()

len(test_set['bush'].unique())


2

Great, now let's do some basic exploratory analysis on the polling data. Let's construct the Bush support for state-age cohorts, and save that data for use in the dictionary, container.

In [12]:
import numpy as np
container = {}; bush_quotient = 0

for age in pre_poll['age'].unique():     # Spanning through all year-state combinations
    for state in pre_poll['state'].unique(): 
        c1 = pre_poll['state'] == state
        c2 = pre_poll['age'] == age
        subset_interest = pre_poll[c1 & c2]  # Subsetting the data 
        
        contain = subset_interest['bush'].unique()  
        contain = contain[~np.isnan(contain)]   # Remove the NaNs - Think why can't we use dropna()?
        
        if len(contain) == 2:
            #Computing the Quotient that polled for Bush in the pre-poll for a particular age-state combination
            bush_quotient = float(subset_interest['bush'].value_counts()[1])/(float(subset_interest['bush'].value_counts()[1])+float(subset_interest['bush'].value_counts()[0]))
            temp_name = str(state)+','+str(age)
            container[temp_name] = bush_quotient

            
# Likewise, let's also produce a dictionary with the general poll percentages for state (no age)

container2 ={}
for state in pre_poll['state'].unique(): 
    c1 = pre_poll['state'] == state
    subset_interest = pre_poll[c1]  # Subsetting the data 
    contain = subset_interest['bush'].unique()  
    contain = contain[~np.isnan(contain)]   # Remove the NaNs - Think why can't we use dropna()?
        
    if len(contain) == 2:
        bush_quotient = float(subset_interest['bush'].value_counts()[1])/(float(subset_interest['bush'].value_counts()[1])+float(subset_interest['bush'].value_counts()[0]))
        temp_name = str(state)
        container2[temp_name] = bush_quotient

You can also probably do something similar to the method above by utilizing the groupby method in Pandas

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html

If you'd like extra practice, please go ahead and recreate the above with this simple pandas method

In [15]:
# Write your optional procedure here

Let's just check out what # 1 shows (all ages)

In [13]:
container2['1']


0.710691823899371

In [14]:
# We also have the actual Election 88 sample percentages to compare with the polling. Let's load the data just to have a look
import pandas as pd

election88_dat = pd.read_csv('election88.csv')

election88_dat.head()


,Unnamed: 0,stnum,st,electionresult,samplesize,raking,_merge
0,1,1,AL,0.59,203,0.673067,3
1,2,2,AK,0.60,0,NaN,1
2,3,3,AZ,0.60,194,0.568980,3
3,4,4,AR,0.56,121,0.563672,3
4,5,5,CA,0.51,1493,0.531725,3


So the electoral data shows that the actual results were .59, whereas the polling showed .71. Does that mean Bush somehow lost almost 17% of his support from the time of the poll to the election? We can put some clarify on this question by doing a simple Bayesian computation

# Combining The Prior and Likelihood 

We see from the table way back at the top that using Beta-Binomial Prior-Likelihood combination allows us to remove the denominator/scaling term, and makes the right-hand side of the Bayesian formula analytically tractable (we can use a Riemann or 'normal' Integral or a simple summation) to compute the probability. Wonderful, polling data is naturally contextualized using Binomial distribution! 

So let's test whether or not $\theta \le .65$, that would basically be the following closed form equation : 
$$P(\theta \le .65) = \frac{\Gamma( 404 )}{\Gamma(141)\Gamma (263)} \int_{0}^{.65} \theta^{144}(1-\theta)^{59}\theta^{119}(1-\theta)^{82}$$

Note: A closed form equation is just a fancy way to mean you can write write something as A = **blah**, where **blah** is a formula that can be computed and is not infinite. 


In [15]:
import scipy.integrate as integrate
from scipy.stats import beta


# UN comment to get the error 
#1/my_beta(141, 263)
def func1(x):
    return (x)**263*(1-x)**141


integrate.quad(lambda x: func1(x) ,0, .65)[0]



9.445274979232658e-116

Uh oh, doesn't work.... what happened? Well, the gamma funciton is not computaitonally tractable. We'll need to figure out, A different way to compute Beta..... what can that be? 

What does tractability mean in data science/computer science? First let's look deeper at my_beta above. Note we use the method gamma to construct beta. This method refers to the gamma **function** and not the distribution. What is gamma? 

Please read the following:
http://mathworld.wolfram.com/GammaFunction.html

Note $\Gamma(a)=(a-1)!$, and if you recall (a-1)! = (a-1)(a-2)...(2)(1). For fun, try to calculate $\Gamma(90)$, $\Gamma(100)$, anything funny happen? Overflow you say? Yep. Gamma explodes very quickly. This is bad for computers (or more precisely Von-Neumann computers -which we all currently use). 

Therefore, we need to find some trick around this explosion. What about logs? Take My_beta and take the log (something you should have had some practice with so far). 

In [16]:
def log_beta(a,b):
    beta_num = math.exp(math.lgamma(a) + math.lgamma(b) - math.lgamma(a+b))
    return beta_num


In [17]:
log_beta(141,263)*integrate.quad(lambda x: func1(x) ,0, .65)[0]


8.005670778380035e-230

# Conclusion and Towards The Future

We see that the $P(\theta \le .65)$ is very close to 0, and therefore we believe the poll did not provide "good" information with respect to the actual electoral results 